# Creación Dataset Venues (Uri)

### Variables a trabajar
Originales:
- is_tw_mp_listed
- marketplace_payment_method
- widget_payment_method

Nuevas:
- appointments_l30d (direct_appointments_l30d + online_appointments_l30d)
- online_appointments_rate (online_appointments_l30d / appointments_l30d)

## Importar librerías

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
import warnings

from google.colab import drive
drive.mount('/content/drive')

from sklearn.preprocessing import OneHotEncoder

Mounted at /content/drive


## Cargar datos

In [2]:
# Cargar el documento y crear el DataFrame
df_raw = pd.read_csv("/content/drive/Shareddrives/Data_TFM/venues_2023_22_21.csv", sep="	", decimal=".", na_values=['(null)'], low_memory=False)
df_raw.head()

,date_day,unique_venue_id,created,venue_status,venue_active_from,original_marketplace_venue_type_name,country_code,tier,saas_product,first_live_date,...,plan_active_from,plan_active_to,prepay,prepaid_until_to,discount,plan_discount_end_date,bill_every,plan_name,plan_fee_eur,is_zero_commission_plan
0,2021-07-01,11000000016,2008-03-25 12:47:09,DISABLED,NaN,Hotel Spa,GB,NaN,tw_connect,2009-07-20 00:00:00,...,2021-04-14 13:31:29,2999-12-31 00:00:00,0,NaN,0,NaN,1.0,Entry,16.65,False
1,2021-08-01,11000000016,2008-03-25 12:47:09,DISABLED,NaN,Hotel Spa,GB,NaN,tw_connect,2009-07-20 00:00:00,...,2021-04-14 13:31:29,2999-12-31 00:00:00,0,NaN,0,NaN,1.0,Entry,16.65,False
2,2021-09-01,11000000016,2008-03-25 12:47:09,DISABLED,NaN,Hotel Spa,GB,NaN,tw_connect,2009-07-20 00:00:00,...,2021-04-14 13:31:29,2999-12-31 00:00:00,0,NaN,0,NaN,1.0,Entry,16.65,False
3,2021-10-01,11000000016,2008-03-25 12:47:09,DISABLED,NaN,Hotel Spa,GB,NaN,tw_connect,2009-07-20 00:00:00,...,2021-04-14 13:31:29,2999-12-31 00:00:00,0,NaN,0,NaN,1.0,Entry,16.65,False
4,2021-11-01,11000000016,2008-03-25 12:47:09,DISABLED,NaN,Hotel Spa,GB,NaN,tw_connect,2009-07-20 00:00:00,...,2021-04-14 13:31:29,2999-12-31 00:00:00,0,NaN,0,NaN,1.0,Entry,16.65,False


## Procesar datos

In [3]:
# Seleccionar las variables a estudiar
df = df_raw.loc[:, ["unique_venue_id", "date_day","direct_appointments_l30d", "online_appointments_l30d", "is_tw_mp_listed",
                    "marketplace_payment_method", "widget_payment_method"]]

# Visualizar el DataFrame
print(df.info())
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4606923 entries, 0 to 4606922
Data columns (total 7 columns):
 #   Column                      Dtype  
---  ------                      -----  
 0   unique_venue_id             int64  
 1   date_day                    object 
 2   direct_appointments_l30d    float64
 3   online_appointments_l30d    float64
 4   is_tw_mp_listed             object 
 5   marketplace_payment_method  object 
 6   widget_payment_method       object 
dtypes: float64(2), int64(1), object(4)
memory usage: 246.0+ MB
None


,unique_venue_id,date_day,direct_appointments_l30d,online_appointments_l30d,is_tw_mp_listed,marketplace_payment_method,widget_payment_method
0,11000000016,2021-07-01,NaN,NaN,True,ALL_METHODS,ALL_METHODS
1,11000000016,2021-08-01,NaN,NaN,True,ALL_METHODS,ALL_METHODS
2,11000000016,2021-09-01,NaN,NaN,True,ALL_METHODS,ALL_METHODS
3,11000000016,2021-10-01,NaN,NaN,True,ALL_METHODS,ALL_METHODS
4,11000000016,2021-11-01,NaN,NaN,True,ALL_METHODS,ALL_METHODS


In [4]:
# Visualizar el porcentaje de NaNs por variable
df.isna().mean()*100

unique_venue_id                0.000000
date_day                       0.000000
direct_appointments_l30d      22.859119
online_appointments_l30d      22.859119
is_tw_mp_listed               12.009968
marketplace_payment_method    12.009968
widget_payment_method         12.009968
dtype: float64

In [5]:
# Modificar el type de las variables
df["unique_venue_id"] = df["unique_venue_id"].astype(str)
df['date_day'] = pd.to_datetime(df['date_day'])
df['direct_appointments_l30d'] = df['direct_appointments_l30d'].astype(pd.Int64Dtype())
df["online_appointments_l30d"] = df["online_appointments_l30d"].astype(pd.Int64Dtype())
df['is_tw_mp_listed'] = df['is_tw_mp_listed'].replace({'true': "True", 'false': "False"}).astype('string')
df["marketplace_payment_method"] = df["marketplace_payment_method"].astype('string')
df["widget_payment_method"] = df["widget_payment_method"].astype('string')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4606923 entries, 0 to 4606922
Data columns (total 7 columns):
 #   Column                      Dtype         
---  ------                      -----         
 0   unique_venue_id             object        
 1   date_day                    datetime64[ns]
 2   direct_appointments_l30d    Int64         
 3   online_appointments_l30d    Int64         
 4   is_tw_mp_listed             string        
 5   marketplace_payment_method  string        
 6   widget_payment_method       string        
dtypes: Int64(2), datetime64[ns](1), object(1), string(3)
memory usage: 254.8+ MB


In [6]:
# Visualizar el porcentaje de NaNs por variable
df.isna().mean()*100

unique_venue_id                0.000000
date_day                       0.000000
direct_appointments_l30d      22.859119
online_appointments_l30d      22.859119
is_tw_mp_listed               12.009968
marketplace_payment_method    12.009968
widget_payment_method         12.009968
dtype: float64

## Trabajar duplicados

In [7]:
# Verificar si existen duplicados
df.duplicated(subset=['unique_venue_id', 'date_day'], keep=False).sum()

344209

In [8]:
# Verificar
comparison = (df.duplicated(subset=['unique_venue_id', 'date_day'], keep=False) == df.duplicated(subset=df.columns, keep=False)).all()
comparison

True

In [9]:
# Eliminar los duplicados
df = df.drop_duplicates(subset=['unique_venue_id', 'date_day'], keep='first')
df.duplicated(subset=df.columns, keep=False).sum()

0

## Codificar variables

In [10]:
# Codificar las variables categóricas y boleanas
onehot_encoder = OneHotEncoder(sparse_output=False)
df[["is_tw_mp_listed", "marketplace_payment_method", "widget_payment_method"]].fillna("N/A", inplace=True)
onehot_encoded = onehot_encoder.fit_transform(df.loc[:, ["is_tw_mp_listed", "marketplace_payment_method",
                                                         "widget_payment_method"]])

# Crear el nuevo DataFrame
df2 = pd.DataFrame(onehot_encoded, columns=onehot_encoder.get_feature_names_out())
df2.head()

<ipython-input-10-24059d21ae43>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[["is_tw_mp_listed", "marketplace_payment_method", "widget_payment_method"]].fillna("N/A", inplace=True)


,is_tw_mp_listed_False,is_tw_mp_listed_N/A,is_tw_mp_listed_True,marketplace_payment_method_ALL_METHODS,marketplace_payment_method_N/A,marketplace_payment_method_PAY_AT_VENUE_ONLY,marketplace_payment_method_PREPAY_ONLY,marketplace_payment_method_UNKNOWN,widget_payment_method_ALL_METHODS,widget_payment_method_N/A,widget_payment_method_PAY_AT_VENUE_ONLY,widget_payment_method_PREPAY_ONLY,widget_payment_method_UNKNOWN
0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
2,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
3,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
4,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


## Crear nuevas variables

In [11]:
# Crear la variable appointments_l30d
df["appointments_l30d"] = df["direct_appointments_l30d"] + df["online_appointments_l30d"]

# Crear la variable online_appointments_rate
df["online_appointments_rate"] = round(df["online_appointments_l30d"]/df["appointments_l30d"], 2)
df.loc[df["online_appointments_l30d"] == 0, "online_appointments_rate"] = 0

# Eliminar las otras variables
df = df.drop("direct_appointments_l30d", axis=1)
df = df.drop("online_appointments_l30d", axis=1)
df.head()

,unique_venue_id,date_day,is_tw_mp_listed,marketplace_payment_method,widget_payment_method,appointments_l30d,online_appointments_rate
0,11000000016,2021-07-01,True,ALL_METHODS,ALL_METHODS,<NA>,<NA>
1,11000000016,2021-08-01,True,ALL_METHODS,ALL_METHODS,<NA>,<NA>
2,11000000016,2021-09-01,True,ALL_METHODS,ALL_METHODS,<NA>,<NA>
3,11000000016,2021-10-01,True,ALL_METHODS,ALL_METHODS,<NA>,<NA>
4,11000000016,2021-11-01,True,ALL_METHODS,ALL_METHODS,<NA>,<NA>


## Crear dataframe final

In [12]:
# Unificar los DataFrames
df2 = df2.set_index(df.index)
df_final = pd.concat([df,df2], axis=1)

# Eliminar las columnas no necesarias
df_final.drop(columns=["is_tw_mp_listed", "marketplace_payment_method", "widget_payment_method"], inplace=True)
df_final.sample(10)

,unique_venue_id,date_day,appointments_l30d,online_appointments_rate,is_tw_mp_listed_False,is_tw_mp_listed_N/A,is_tw_mp_listed_True,marketplace_payment_method_ALL_METHODS,marketplace_payment_method_N/A,marketplace_payment_method_PAY_AT_VENUE_ONLY,marketplace_payment_method_PREPAY_ONLY,marketplace_payment_method_UNKNOWN,widget_payment_method_ALL_METHODS,widget_payment_method_N/A,widget_payment_method_PAY_AT_VENUE_ONLY,widget_payment_method_PREPAY_ONLY,widget_payment_method_UNKNOWN
2986241,11000278451,2022-05-01,0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
3631566,11000412521,2021-11-01,<NA>,<NA>,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
495779,11000294535,2022-10-01,0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
3694657,11000206812,2022-12-01,0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
360656,11000287509,2021-10-01,<NA>,<NA>,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1359559,11000369459,2022-05-01,119,0.42,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
819514,11000309097,2021-07-01,<NA>,<NA>,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
3096458,11000023394,2022-12-01,0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
898475,11000316926,2021-07-01,<NA>,<NA>,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1535272,11000377867,2021-11-01,<NA>,<NA>,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0


In [13]:
# Verificar que no existen filas duplicadas
df_final.duplicated(subset=['unique_venue_id', 'date_day'], keep=False).sum()

0

In [14]:
print(df_raw['direct_appointments_l30d'].isna().sum())
print(df_raw['online_appointments_l30d'].isna().sum())
print(df['appointments_l30d'].isna().sum())
print(df['online_appointments_rate'].isna().sum())
print(df_final['appointments_l30d'].isna().sum())
print(df_final['online_appointments_rate'].isna().sum())

1053102
1053102
1053071
1053071
1053071
1053071


## Crear CSV

In [15]:
#df_final.to_csv('/content/drive/Shareddrives/Data_TFM/uri_venues.csv', sep="	", index=False)